# 📊 Data Comparison: Training vs Production

**Purpose:** Detect data drift between training and production datasets.

**Why Important:**
- Model trained on training data distribution
- Production data might have different distribution
- Drift = predictions might be unreliable

**What We Check:**
1. Statistical differences (mean, std)
2. Distribution differences (range, percentiles)
3. Per-sensor drift
4. Data quality issues

In [1]:
import pandas as pd

In [7]:
with_activities = pd.read_csv("all_users_data_labeled.csv")



In [8]:
with_activities.head()

,timestamp,Ax_w,Ay_w,Az_w,Gx_w,Gy_w,Gz_w,activity,User
0,2005-05-01 00:04:59.000,8.500479,-4.885620,-1.247403,-18.151813,1.957602,-0.362040,ear_rubbing,1
1,2005-05-01 00:04:59.020,8.628770,-4.715455,-1.247403,-20.759232,4.337617,-1.517069,ear_rubbing,1
2,2005-05-01 00:04:59.040,8.571752,-4.521656,-0.966979,-18.816491,4.285163,-1.692155,ear_rubbing,1
3,2005-05-01 00:04:59.060,8.581255,-4.436574,-0.962144,-16.541813,4.180102,-1.744540,ear_rubbing,1
4,2005-05-01 00:04:59.080,8.310418,-4.394033,-0.957309,-22.858267,3.042800,0.792587,ear_rubbing,1


In [11]:
without_activities = pd.read_csv("sensor_fused_50Hz.csv")
without_activities.head()

,timestamp_ms,timestamp_iso,Ax,Ay,Az,Gx,Gy,Gz
0,1742802739780,2025-03-24T07:52:19.780000Z,-95.41802,-1240.5320,-1086.74200,39.156140,-7.520049,7.881836
1,1742802739800,2025-03-24T07:52:19.800000Z,-159.83730,-1364.8455,-883.68655,27.501250,-10.197520,10.262000
2,1742802739820,2025-03-24T07:52:19.820000Z,-161.29030,-1311.8440,-529.81760,1.041086,-4.790063,2.981754
3,1742802739840,2025-03-24T07:52:19.840000Z,-123.02625,-1157.1745,-302.61215,-21.761370,-7.012552,0.549316
4,1742802739860,2025-03-24T07:52:19.860000Z,-149.66580,-806.8806,-136.52045,-44.003680,-6.960001,-3.230397


In [12]:
with_activities.describe()


,Ax_w,Ay_w,Az_w,Gx_w,Gy_w,Gz_w,User
count,385326.000000,385326.000000,385326.000000,385326.000000,385326.000000,385326.000000,385326.000000
mean,3.218580,1.282084,-3.528949,0.599312,0.225205,0.088668,3.529199
std,6.568341,4.351470,3.236169,49.930271,14.811729,14.166785,1.712415
min,-26.846118,-16.560816,-45.230434,-818.951569,-302.422165,-242.407539,1.000000
25%,-1.531575,-1.586314,-6.077462,-6.113840,-2.717356,-2.910012,2.000000
50%,6.381298,2.209306,-2.954120,0.672039,0.235012,0.060095,4.000000
75%,8.757061,4.118933,-1.503652,7.454204,3.034424,2.979887,5.000000
max,31.954008,41.857702,24.150296,835.172150,206.932671,231.835464,6.000000


In [13]:
without_activities.describe()

,timestamp_ms,Ax,Ay,Az,Gx,Gy,Gz
count,1.816990e+05,181673.000000,181673.000000,181673.000000,181673.000000,181673.000000,181673.000000
mean,1.742805e+12,-16.234198,-19.022889,-1001.556902,0.492359,0.150784,0.131591
std,1.049043e+06,11.315276,31.041663,19.924188,5.332085,4.716818,1.873494
min,1.742803e+12,-1455.487750,-1924.737350,-2833.415500,-800.861450,-584.833433,-143.265950
25%,1.742804e+12,-17.921175,-21.007985,-1002.957500,0.446533,0.127297,0.112309
50%,1.742805e+12,-16.468110,-19.241257,-1001.971500,0.498668,0.145051,0.146600
75%,1.742805e+12,-15.015045,-14.744130,-1000.986000,0.550757,0.179977,0.180059
max,1.742806e+12,955.633000,1263.563500,390.832900,448.901100,582.544950,253.459250


## 📖 Understanding Data Drift Parameters

### What is Data Drift?

**Definition:** When production data distribution differs significantly from training data.

**Example:**
```
Training: Users aged 20-30, flat walking
Production: Users aged 60-70, stairs
→ Different patterns! Model might fail.
```

### Key Drift Metrics

#### 1. Mean Drift
**Formula:**
```python
drift_mean = |production_mean - training_mean|
```

**Threshold:** drift > 0.1 indicates significant shift

**Example:**
```
Training Az mean: 9.81 m/s²
Production Az mean: 10.2 m/s²
Drift: |10.2 - 9.81| = 0.39 ⚠️ HIGH DRIFT
```

**What it means:**
- Data is systematically shifted (bias)
- Sensors might be calibrated differently
- Different user behavior patterns

#### 2. Standard Deviation Drift
**Formula:**
```python
drift_std = |production_std - training_std|
```

**Threshold:** drift > 0.15 indicates variability change

**Example:**
```
Training Az std: 0.50
Production Az std: 0.85
Drift: |0.85 - 0.50| = 0.35 ⚠️ HIGH DRIFT
```

**What it means:**
- Data variability has changed
- More/less dynamic movements
- Different activity intensity

#### 3. Range Drift
**Check if production values exceed training range:**

**Example:**
```
Training range: [-12, 15]
Production range: [-18, 25]
→ Production has values outside training!
```

**Risk:** Model extrapolating (unreliable predictions)

#### 4. Per-Sensor Drift

Check each sensor separately:

| Sensor | Train Mean | Prod Mean | Drift | Status |
|--------|-----------|-----------|-------|--------|
| Ax | 0.12 | 0.17 | 0.05 | ✅ OK |
| Ay | -0.08 | -0.05 | 0.03 | ✅ OK |
| Az | 9.81 | 10.2 | 0.39 | ⚠️ HIGH |
| Gx | 0.05 | 0.13 | 0.08 | ✅ OK |
| Gy | -0.02 | 0.50 | 0.52 | ⚠️ HIGH |
| Gz | 0.01 | 0.10 | 0.09 | ✅ OK |

### Drift Thresholds Summary

| Metric | Threshold | Action |
|--------|-----------|--------|
| **Mean drift** | > 0.1 | ⚠️ Investigate |
| **Std drift** | > 0.15 | ⚠️ Check variability |
| **Range outside** | Any | ⚠️ Extrapolation risk |
| **KS p-value** | < 0.05 | ⚠️ Different distributions |

### What to Do When Drift Detected

✅ **Low drift (< thresholds):** Proceed with inference

⚠️ **Medium drift (near thresholds):**
- Monitor predictions closely
- Validate on sample data
- Check prediction confidence

❌ **High drift (>> thresholds):**
- **Investigate cause:**
  - Different users?
  - Different devices?
  - Different activities?
- **Actions:**
  - Collect more training data from production
  - Retrain model with combined data
  - Recalibrate sensors
  - Adjust preprocessing

### Example Interpretation

**Scenario:**
```
Ax drift: 0.05  ✅ OK
Ay drift: 0.03  ✅ OK
Az drift: 0.45  ❌ HIGH
Gx drift: 0.08  ✅ OK
Gy drift: 0.52  ❌ HIGH
Gz drift: 0.09  ✅ OK
```

**Analysis:**
- Accelerometer Z-axis drifted (maybe gravity sensor issue)
- Gyroscope Y-axis drifted (maybe rotation pattern different)
- Other sensors OK

**Action:**
- Check Az calibration
- Investigate Gy activities (turning motions?)
- Consider retraining model